# **Productor:**

In [51]:
!pip install kafka-python

In [52]:
!curl -sSOL https://dlcdn.apache.org/kafka/3.6.2/kafka_2.13-3.6.2.tgz
!tar -xzf kafka_2.13-3.6.2.tgz

In [53]:
!./kafka_2.13-3.6.2/bin/zookeeper-server-start.sh -daemon ./kafka_2.13-3.6.2/config/zookeeper.properties
!./kafka_2.13-3.6.2/bin/kafka-server-start.sh -daemon ./kafka_2.13-3.6.2/config/server.properties
!echo "Esperamos 10 segundos hasta que los servicios kafka y zookeeper estén activos y funcionando"
!sleep 10

Esperamos 10 segundos hasta que los servicios kafka y zookeeper estén activos y funcionando


In [54]:
!ps -ef | grep kafka

root       19423     241  0 07:59 ?        00:00:00 /bin/bash -c ps -ef | grep kafka
root       19425   19423  0 07:59 ?        00:00:00 grep kafka


In [55]:
!./kafka_2.13-3.6.2/bin/kafka-topics.sh --create --topic temperaturas --bootstrap-server localhost:9092

Created topic temperaturas.


In [56]:
!./kafka_2.13-3.6.2/bin/kafka-topics.sh --describe --bootstrap-server 127.0.0.1:9092  --topic temperaturas

Topic: temperaturas	TopicId: clLVhPTcT6yvf-JG5nzsuQ	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: temperaturas	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [57]:
nombre_topico='temperaturas'

In [58]:
!cat producer.py

#producer.py
from kafka import KafkaProducer
import time
import json
import pandas as pd
import sys
import random

class Sensor:
    def __init__(self):
        self.data = 0

    def set_data(self):
        t = random.randrange(0, 50)
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        self.data = [timestamp, t]


class Producer:
    def __init__(self, topic, freq):
        self.topic = topic
        self.freq = freq if isinstance(freq, int) else int(freq)  #freq es la frecuencia con la que se envían (producen) los mensajes, que es 5 seg en nuestro caso
        self.producer = KafkaProducer(bootstrap_servers='localhost:9092',
                                      value_serializer=lambda x: json.dumps(x).encode('utf-8'))

        self.sensor = Sensor()
        self.num_messages = 0

    def start_write(self):
        while True:
            self.num_messages+=1
            self.sensor.set_data()
            value = self.sensor.data
            self.producer

In [59]:
!nohup python -m producer temperaturas 5 &

nohup: appending output to 'nohup.out'


In [60]:
!./kafka_2.13-3.6.2/bin/kafka-console-consumer.sh --topic temperaturas --from-beginning --bootstrap-server localhost:9092

["2024-05-09 07:59:42", 28]
["2024-05-09 07:59:47", 42]
["2024-05-09 07:59:52", 5]
["2024-05-09 07:59:57", 18]
["2024-05-09 08:00:02", 38]
Processed a total of 5 messages


# **Consumidor:**

In [62]:
from kafka import KafkaConsumer
import json



class Consumer:
    def __init__(self, topic):
        self._consumer = KafkaConsumer(topic,
                                       bootstrap_servers='localhost:9092',
                                       value_deserializer=lambda x: json.loads(x.decode('utf-8')))

        self.data = []

    @property
    def consumer(self):
        return self._consumer

    @consumer.setter
    def consumer(self, value):
        if isinstance(value, KafkaConsumer):
            self._consumer = value

    def iniciar_consumicion(self):
        self.receive_message()

    def receive_message(self):
        message_count = 0
        for message in self._consumer:
            message = message.value
            print(f'Message {message_count}: {message}')
            self.data.append(message)
            message_count += 1

In [63]:
consumer = Consumer(nombre_topico)
consumer.iniciar_consumicion()

Message 0: ['2024-05-09 08:00:27', 16]
Message 1: ['2024-05-09 08:00:32', 35]
Message 2: ['2024-05-09 08:00:37', 44]
Message 3: ['2024-05-09 08:00:42', 19]


KeyboardInterrupt: 

# Una vez tenemos implementados el consumidor y el productor, lo introducimos en nuestro programa principal (una vez creado anteriormente el tópico 'temperaturas'):

In [64]:
from abc import ABC, abstractmethod
import time
import random
import functools
import numpy as np
from kafka import KafkaProducer
import json
import pandas as pd
import sys

"""
La idea es que nuestro sensor es el sujeto observable, el que produce los datos, mientras que el sistema es el observador, el que recibe los datos del sensor.
Solo hay una instancia del Sistema, y además, cuando recibe un nuevo dato, este dato lo pasa a otras clases (chain of responsability) que hagan ya las operaciones.

"""

# -----------------------
# EXCEPCIONES
# -----------------------
class ErrorEnTemperatura(Exception):
    pass

class ErrorEnObservador(Exception):
    pass

class ErrorEnEstrategia(Exception):
    pass

class ErrorEnCadena(Exception):
    pass



# -----------------------
# CLASES
# -----------------------



class Sensor:
    def __init__(self, name):
        super().__init__()
        self.data = 0

    def set_data(self):
        t = random.randrange(0, 50)
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
        self.data = [timestamp, t]


class Producer: #PRODUCTOR
    def __init__(self, topic, freq):
        self.topic = topic
        self.freq = freq if isinstance(freq, int) else int(freq)  #freq es la frecuencia con la que se envían (producen) los mensajes, que es 5 seg en nuestro caso
        self.producer = KafkaProducer(bootstrap_servers='localhost:9092',
                                      value_serializer=lambda x: json.dumps(x).encode('utf-8'))

        self.sensor = Sensor()
        self.num_messages = 0

    def start_write(self):
        while True:
            self.num_messages+=1
            self.sensor.set_data()
            value = self.sensor.data
            self.producer.send(self.topic, value=value)
            print(f'Message {self.num_messages}: {value}')
            time.sleep(self.freq)




class Sistema:     #CONSUMIDOR
    _unicaInstancia = None

    def __init__(self,topic):
      self._consumer = KafkaConsumer(topic,
                                       bootstrap_servers='localhost:9092',
                                       value_deserializer=lambda x: json.loads(x.decode('utf-8')))

      self.data = []
      self._name = "Sistema_temperatura" #Nombre de nuestro sistema
      self._estrategia=None


    @property
    def consumer(self):
        return self._consumer


    @consumer.setter
    def consumer(self, value):
        if isinstance(value, KafkaConsumer):
            self._consumer = value


    def iniciar_consumicion(self):
      self.receive_message()

    @classmethod
    def obtener_instancia(cls,topico) :
        if not cls._unicaInstancia :
            cls._unicaInstancia = cls(topico)
        return cls._unicaInstancia

    def receive_message(self):  #el error en cuanto al tipo de temperatura ya es tratado en la clase 'Sensor'
        message_count=0
        for message in self._consumer :
          message = message.value
          print ("Nueva temperatura {} - Timestamp: {}, Temperatura: {}".format(message_count,message[0],message[1]))
          self.data.append(message)
          message_count += 1
          e = Estadísticos()   #CHAIN OF RESPONSABILITY
          e.establecer_estrategia(self._estrategia)  #Se establece la estrategia a tomar en la clase 'Estadísticos'
          u = UmbralTemperatura(e)
          a = AumentoTemperatura(u)
          t = message[1]
          a.handle_request(t)

    def establecer_estrategia(self,estrategia) :
        if isinstance(estrategia,Estrategia1) or isinstance(estrategia,Estrategia2) or isinstance(estrategia,Estrategia3) :
            self._estrategia = estrategia
        else :
            print("La estrategia debe ser del tipo 1, 2 o 3")
            raise ErrorEnEstrategia









#CHAIN OF RESPONSABILITY, donde la temperatura pasa por todas las clases obligatoriamente
class Manejador(ABC) :
    def __init__(self,sucesor=None) :
        self.sucesor = sucesor

    def handle_request(self,temperatura) :
        pass





class Estadísticos(Manejador) :    #Esta es nuestra clase para aplicar STRATEGY
    def __init__(self,sucesor=None,estrategia=None,lista_temperaturas_e=[]) :
        if sucesor is not None and not isinstance(sucesor,UmbralTemperatura) and not isinstance(sucesor,AumentoTemperatura) :
            print("Las operaciones a realizar son AumentoTemperatura, UmbralTemperatura y Estadísticos.")
            raise ErrorEnCadena
        super().__init__(sucesor)
        self.estrategia = estrategia
        self.lista_temperaturas_e = lista_temperaturas_e
    def establecer_estrategia(self,estrategia_concreta) : #El error del tipo de estrategia ya es tratado en la clase 'Sistema'
        self.estrategia = estrategia_concreta
    def handle_request(self,temperatura) :
        self.lista_temperaturas_e.append(temperatura)
        if len(self.lista_temperaturas_e) == 12:
            print(f"Cálculo de estadísticos en base a las temperaturas de los últimos 60 segundos: ")
            self.estrategia.aplicarAlgoritmo(self.lista_temperaturas_e)
            self.lista_temperaturas_e.clear() #se eliminan todos las temperaturas de los últimos 60 segundos una vez se han calculado sus estadísticos

        #Ahora pasamos al siguiente de la cadena:
        if self.sucesor :
            self.sucesor.handle_request(temperatura)


class UmbralTemperatura(Manejador) :
    def __init__(self,sucesor=None,lista_temperaturas_u = []) :
        if sucesor is not None and not isinstance(sucesor,Estadísticos) and not isinstance(sucesor,AumentoTemperatura) :
            print("Las operaciones a realizar son AumentoTemperatura, UmbralTemperatura y Estadísticos.")
            raise ErrorEnCadena
        super().__init__(sucesor)
        self.lista_temperaturas_u = lista_temperaturas_u
    def handle_request(self,temperatura) : #FIJAMOS EL UMBRAL A 27ºC
        self.lista_temperaturas_u.append(temperatura)
        if temperatura > 27 :
            print("Temperatura actual por encima de 27ºC") #SE REALIZA LA OPERACIÓN DE COMPROBAR SI LA TEMPERATURA ACTUAL DEL INVERNADERO ESTÁ POR ENCIMA DEL UMBRAL QUE FIJEMOS
        #Ahora pasamos al siguiente de la cadena:
        if self.sucesor :
            self.sucesor.handle_request(temperatura)


class AumentoTemperatura(Manejador) :
    def __init__(self,sucesor=None,lista_temperaturas_a = []) :
        if sucesor is not None and not isinstance(sucesor,UmbralTemperatura) and not isinstance(sucesor,Estadísticos) :
            print("Las operaciones a realizar son AumentoTemperatura, UmbralTemperatura y Estadísticos.")
            raise ErrorEnCadena
        super().__init__(sucesor)
        self.lista_temperaturas_a = lista_temperaturas_a
    def handle_request(self,temperatura) :
        self.lista_temperaturas_a.append(temperatura)
        if len(self.lista_temperaturas_a) == 6:
            sol = list(filter(lambda x: x>10,self.lista_temperaturas_a))
            if len(sol) > 0 :
                print("La temperatura ha aumentado más de 10 grados en los últimos 30 segundos.")
            self.lista_temperaturas_a.clear() #una vez han pasado los 30 segundos (hay 6 temperaturas registradas), la lista se establece de nuevo a cero.
        #SE REALIZA LA OPERACIÓN DE comprobar si durante los últimos 30 segundos la temperatura ha aumentado más de 10 grados
        #Ahora pasamos al siguiente de la cadena:
        if self.sucesor :
            self.sucesor.handle_request(temperatura)


#ESTRATEGIAS DE STRATEGY:
class Estrategia(ABC) :
    def aplicarAlgoritmo(self,T) :
        pass


class Estrategia1(Estrategia) :
    def aplicarAlgoritmo(self,T) :
        media = functools.reduce(lambda x,y:x+y,T)/len(T)
        des = np.sqrt(sum(map(lambda x: ((x-media)**2),T))/(len(T)-1))
        print(f"\n\tCálculo de la media y desviación típica: \n\t|\tMedia: {round(media,2)}\n\t|\tDesviación típica: {round(des,2)}\n")
        #CÁLCULO DE LA MEDIA Y DESVIACIÓN TÍPICA CON LA TEMPERATURA


class Estrategia2(Estrategia) :
    def aplicarAlgoritmo(self,T) :
        print("\tCómputo de cuantiles:\n ")
        T = sorted(T)
        cuartil1 = round((T[3]+T[4])/2,0)  #La longitud de T siempre es 12
        cuartil2 = round((T[5]+T[6])/2,0)
        cuartil3 = round((T[9]+T[10])/2,0)
        print(f"\t\tCuartiles: {cuartil1},{cuartil2},{cuartil3}\n") #Calculamos los cuartiles
        #CALCULAMOS LOS CUANTILES DE LA TEMPERATURA


class Estrategia3(Estrategia) :
    def aplicarAlgoritmo(self,T) :
        maximo = lambda a,b: a if (a>b) else b
        minimo = lambda a,b: a if (a<b) else b
        max = functools.reduce(maximo,T)
        min = functools.reduce(minimo,T)
        print(f"\n\tValores máximos y mínimos en un periodo de 60 segundos: máximo:{max}, mínimo:{min}\n")
        #CALCULAMOS LOS VALORES MÁXIMOS Y MÍNIMOS EN UN PERIODO DE 60 SEGUNDOS





if __name__ == "__main__" :
    nombre_topico = "temperaturas"
    consumer = Sistema.obtener_instancia(nombre_topico)
    consumer = Sistema(nombre_topico)
    estrategia1 = Estrategia1()    #instanciamos la estrategia que queramos
    consumer.establecer_estrategia(estrategia1)
    consumer.iniciar_consumicion()



Nueva temperatura 0 - Timestamp: 2024-05-09 08:00:52, Temperatura: 28
Temperatura actual por encima de 27ºC
Nueva temperatura 1 - Timestamp: 2024-05-09 08:00:57, Temperatura: 6
Nueva temperatura 2 - Timestamp: 2024-05-09 08:01:02, Temperatura: 33
Temperatura actual por encima de 27ºC
Nueva temperatura 3 - Timestamp: 2024-05-09 08:01:07, Temperatura: 37
Temperatura actual por encima de 27ºC
Nueva temperatura 4 - Timestamp: 2024-05-09 08:01:12, Temperatura: 23
Nueva temperatura 5 - Timestamp: 2024-05-09 08:01:17, Temperatura: 5
La temperatura ha aumentado más de 10 grados en los últimos 30 segundos.
Nueva temperatura 6 - Timestamp: 2024-05-09 08:01:22, Temperatura: 19
Nueva temperatura 7 - Timestamp: 2024-05-09 08:01:27, Temperatura: 25
Nueva temperatura 8 - Timestamp: 2024-05-09 08:01:32, Temperatura: 43
Temperatura actual por encima de 27ºC
Nueva temperatura 9 - Timestamp: 2024-05-09 08:01:37, Temperatura: 37
Temperatura actual por encima de 27ºC
Nueva temperatura 10 - Timestamp: 2024-

KeyboardInterrupt: 